In [5]:
import json

In [6]:
import random


def generate_machine(machine_id, base_power, switching_off_power, switching_on_power, sleeping_power, switching_on_time, switching_off_time):
    """Generate one machine entry with DVFS profiles and states."""

    # Add random variations to create heterogeneity (±15%)
    # variation = random.uniform(0.85, 1.15)
    variation = 1
    base_power *= variation
    base_speed = 1.0 * variation  # Vary compute speed too

    # Different machines have different DVFS capabilities
    dvfs_options = [
        # Standard 3-level DVFS
        {
            "underclock_1": {"power": base_power * 0.7, "compute_speed": base_speed * 0.7},
            "base": {"power": base_power, "compute_speed": base_speed},
            "overclock_1": {"power": base_power * 1.3, "compute_speed": base_speed * 1.3},
        },
        # Limited 2-level DVFS (older machines)
        {
            "underclock": {"power": base_power * 0.8, "compute_speed": base_speed * 0.8},
            "base": {"power": base_power, "compute_speed": base_speed},
        },
        # Extended 5-level DVFS (newer machines)
        {
            "underclock_2": {"power": base_power * 0.5, "compute_speed": base_speed * 0.5},
            "underclock_1": {"power": base_power * 0.75, "compute_speed": base_speed * 0.75},
            "base": {"power": base_power, "compute_speed": base_speed},
            "overclock_1": {"power": base_power * 1.25, "compute_speed": base_speed * 1.25},
            "overclock_2": {"power": base_power * 1.5, "compute_speed": base_speed * 1.4},
        }
    ]

    # Randomly choose a DVFS profile set
    dvfs_profiles = random.choice(dvfs_options)

    machine = {
        "id": machine_id,
        "dvfs_profiles": dvfs_profiles,
        "dvfs_mode": "base",
        "states": {
            "active": {
                "power": "from_dvfs",
                "compute_speed": "from_dvfs",
                "can_run_jobs": True,
                "transitions": [{"state": "switching_off", "transition_time": 0}],
            },
            "switching_off": {
                "power": switching_off_power * variation,  # Scale with variation
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "sleeping", "transition_time": switching_off_time}],
            },
            "switching_on": {
                "power": switching_on_power * variation,  # Scale with variation
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "active", "transition_time": switching_on_time}],
            },
            "sleeping": {
                "power": sleeping_power * variation,  # Scale with variation
                "compute_speed": "from_dvfs",
                "can_run_jobs": False,
                "transitions": [{"state": "switching_on", "transition_time": 0}],
            },
        },
    }

    return machine


def generate_cluster(num_nodes):
    # Create different machine types by varying the base parameters
    machines = []
    for machine_id in range(num_nodes):
        # Randomly choose machine type characteristics
        machine_type = random.choice(["standard"])

        if machine_type == "standard":
            machine = generate_machine(machine_id, 190, 9, 190, 9, 1800, 2700)
        elif machine_type == "efficient":
            # Lower power, slower switching
            machine = generate_machine(machine_id, 150, 6, 150, 6, 15, 15)
        else:  # high_perf
            # Higher power, faster switching
            machine = generate_machine(machine_id, 250, 12, 250, 12, 5, 5)

        machines.append(machine)

    return {"machines": machines}

In [7]:
# Number of nodes to generate
num_node = 196
# Path to output JSON
output_path = "platforms/platform-nasa-196-1800son-2700sof-w190son-w9sof.json"


In [8]:

cluster = generate_cluster(num_node)

with open(output_path, "w") as json_file:
    json.dump(cluster, json_file, indent=4)
